In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [ ]:
Q_amount = []
Q_uniques = []
Q_W_amount = []
Ws = {}
trash = 0
for i in range(8):
    Q_W_amount.append([])
    Q_uniques.append([])
P_amount = []
permutations_amount = []
for i in range(3, 11):
    data = pd.read_csv(f'results/results_{i}.csv', delimiter=';')
    permutations_amount.append(len(data["W"]))
#     experiment 1: count unique Q-tableaux
    Q = set(data["Q"])
    Q_amount.append(len(Q))
#     experiment 2: amount of perms for each Q
    Q_uniques[i-3], Q_W_amount[i-3] = np.unique(data["Q"], return_counts=True)
    '''
    Берем все уникальные Q, сохраняем в словарь,
    в соответствие каждому Q записываем перестановку
    '''
    # data = np.array(data)
    # print('np array done')
    # for q in Q_uniques[i-3]:
    #     w_temp = []
    #     for line in data:
    #         if line[3] == q:
    #             w_temp.append(line[0])
    #     Ws.update({q: w_temp})
    print('done' + str(i))

In [ ]:
Q_amount

In [ ]:
Q_W_amount[2]

In [ ]:
import pickle
with open('Q_W.pickle', 'wb') as handle:
    pickle.dump(Ws, handle)

In [ ]:
x = np.arange(3, 11, 1)
y = np.array(Q_amount)
labels = "Кол-во уникальных Q"

In [ ]:
'''
График 1: x - мощность перестановок, y - кол-во уникальных Q
График 2: поиск наиболее близкой к графику 1 функции
'''
plt.figure(figsize=(6, 8), dpi=80)
plt.ylim(0, 10000)
plt.plot(x, y, 'o-', label=labels)
plt.legend(loc='upper left')
plt.grid(True)
x_1 = np.arange(3, 10, 0.01)
plt.plot(x_1, (math.pi ** (x_1 - 2)), '-', label='pi^x')
plt.legend(loc='upper left')
plt.yticks([y[-4], y[-3], y[-2], 4000, 6000, 8000, y[-1]])
plt.savefig("Q_tableaux.png", dpi=80)

In [ ]:
plt.figure(figsize=(6, 8), dpi=80)
plt.plot(x, math.pi ** (x - 2) - y, 'o-', label='pi^x - y')
plt.yticks(math.pi ** (x - 2) - y)
plt.legend(loc='upper left')
plt.grid(True)
x_2 = np.arange(3, 9, 0.01)
plt.plot(x_2, (math.e ** (x_2 - 3)), '-', label='e^x')
plt.legend(loc='upper left')

In [ ]:
'''
График: x - номер таблицы Q, y - кол-во ее повторений
'''
y = [Q_W_amount[i] for i in range(8)]
plt.figure(figsize=(8, 6), dpi=80)
for i in range(3, 4):
    x = np.array([i+1 for i in range(len(y[i]))])
    plt.yticks(y[i])
    # plt.ylim(200)
    plt.plot(x, y[i], 'o', label=f'N={i+3}')
    plt.legend(loc='upper left')
plt.grid(True)
plt.xlabel('Порядковый номер уникальной Q', fontsize=16)
plt.ylabel('Количество перестановок', fontsize=16)
plt.savefig("Q_amounts_6_dots.png", dpi=80)

In [ ]:
'''
Для каждой формы Q подсчитать кол-во повторений
'''
# Experiment 3: how many Q-tableaux for each form
forms = {}
for i in range(8):
    for Q in Q_uniques[i]:
        # dictionary {Q-form: amount of repeats}
        line = 0
        form = ''
        for x in Q:
            if x == ',':
                line += 1
            elif x == 'n':
                form += str(line)
                line = 0
        if form not in forms.keys():
            forms.update({form: 1})
        else:
            forms[form] += 1

In [ ]:
forms

In [ ]:
'''
Для каждой формы Q вывести все перестановки,
у которых Q имеет взятую форму
'''
# Experiment 4: permutations for each form
forms_w = {}
for i in range(8):
    for q in Q_uniques[i]:
        # dictionary {Q-form: permutations}
        line = 0
        form = ''
        for x in q:
            if x == ',':
                line += 1
            elif x == 'n':
                form += str(line)
                line = 0
        if form not in forms_w.keys():
            forms_w.update({form: []})
        for w in Ws[q]:
            forms_w[form].append(w)

In [ ]:
print(forms_w['31'], forms_w['211'], sep='\n')

In [ ]:
print(forms_w['41'], forms_w['2111'], sep='\n')

Ищем классы перестановок:

In [ ]:
print(Ws['1,2,5,n3,4,n'], Ws['1,3,5,n2,n4,n'], sep='\n')

In [ ]:
y = [np.sort(Q_W_amount[i]) for i in range(8)]
plt.figure(figsize=(6, 8), dpi=80)
for i in range(5, 7):
    x = np.array([i+1 for i in range(len(y[i]))])
    plt.yticks(y[i])
    plt.plot(x, y[i], '-', label=f'N={i+3}')
    plt.legend(loc='upper left')
plt.grid(True)
plt.savefig("Q_amounts_8-9_sorted.png", dpi=80)

Теперь считаем для перестановок у которых Q не меняется поле rc

In [ ]:
amounts = []
amounts_unique = [0] * 11
Q_rc_uniques = []
for i in range(11):
    Q_rc_uniques.append([])
for i in range(1, 12):
    data = pd.read_csv(f'results/simil_Q/results_simil_Q_{i}.csv', delimiter=';')
    amounts.append(len(data))
    np.array(data)
    Q_rc_uniques[i-1], amounts_unique[i-1] = np.unique(data["Q"], return_counts=True)
    '''
    проверим, что знаки неравенства
    не меняются после применения rc
    (не проверяем для перестановок с N=1)
    '''
    if i != 1:
        data = np.array(data)
        flag = True
        for line in data:
            w = line[0].split(',')
            w.pop()
            wrc = line[1].split(',')
            wrc.pop()
            for j in range(i-1):
                if ((int(w[j]) > int(w[j+1])) and (int(wrc[j]) < int(wrc[j+1]))) \
                        or ((int(w[j]) < int(w[j+1])) and (int(wrc[j]) > int(wrc[j+1]))):
                    flag = False
                    break
        print(f'N={i} - {flag}')

In [ ]:
print(Q_amount, amounts_unique, sep='\n')

In [ ]:
'''
количество таблиц, у которых она совпадает после применения операции rc
'''
x = np.arange(1, 12, 1)
y = np.array(amounts_unique)
labels = "Кол-во уникальных Q"
plt.figure(figsize=(6, 8), dpi=80)
plt.ylim(0, 350)
plt.yticks(y[3:])
plt.xticks(x)
plt.plot(x, y, 'o-', label=labels)
plt.legend(loc='upper left')
plt.xlabel('Мощность перестановки', fontsize=16)
plt.ylabel('Уникальные таблицы Q', fontsize=16)
plt.grid(True)
plt.savefig("Q_amounts_rc.png", dpi=80)